# PyTorch ONNX Exporter new features and architecture

Jan 2026

![infographics](<pt-onnx-infographics.png>)

## `dynamo=True` is the default

- The New Default: Starting from PyTorch 2.9, the **`dynamo=True`** option is the **default and recommended** way to export models to ONNX.
- Core Shift: It moves away from the older TorchScript-based capture mechanism to a torch.export based modern stack.
- Deprecation Plan: While the TorchScript exporter (dynamo=False) is currently usable, it is planned for eventual deprecation in alignment with PyTorch core's handling of TorchScript.

## New options in `export()`

```py
torch.onnx.export(
    model, args, kwargs=kwargs,
    # New way of expressing dynamic shapes (more examples later)
    dynamic_shapes=({0: "batch", 1: "sequence_len"}),
    # dynamic_axes=...,  # Deprecated
    dynamo=True,  # Default (2.9)
    report=True,  # Creates a markdown report
    verify=True,  # Runs onnx runtime on the example
    optimize=True, # Runs onnxscript graph optimizations
) -> torch.onnx.ONNXProgram
```

## What happens inside `torch.onnx.export`

torch.export() **captures FX** graph -> **translate** and build ONNX IR -> graph **optimization**

Entry point is at: https://github.com/pytorch/pytorch/blob/0ad306cac740eaf2ce582e2bdf097cc61d929a40/torch/onnx/_internal/exporter/_core.py#L1282

![diagram](https://raw.githubusercontent.com/justinchuby/diagrams/refs/heads/main/pytorch/torch-export-flow.svg)

### Metadata saved

https://docs.pytorch.org/docs/stable/onnx_export.html#metadata